In [1]:
!pip install openai
!pip install rouge_metric
!pip install transformers
import nltk
nltk.download("wordnet")
from google.colab import drive
drive.mount('/content/drive')
!cd drive/MyDrive/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 2.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.

[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


In [2]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

import collections
import json
import os
import regex




# will need to change based on path that stores mbpp.jsonl
class MBPPGoogleDataset(object):
    def __init__(self, path='/content/drive/MyDrive/data/mbpp/mbpp.jsonl', mode='function_name'):
        raw_data = sorted([json.loads(x) for x in open(path)], key=lambda x: x['task_id'])
        for i, data_item in enumerate(raw_data):
            assert data_item['task_id'] == i + 1
        self.raw_data = collections.defaultdict()
        self.mode = mode
        # 374 for training, 100 heldout, 500 test
        self.raw_data['train'] = raw_data[:10] + raw_data[510:]
        self.raw_data['test'] = raw_data[10:510]
        # data for codex collector, in input-output-info format
        self.data = collections.defaultdict()
        for split in self.raw_data:
            self.data[split] = self.extract_data(self.raw_data[split], mode)

    @staticmethod
    def extract_data(raw_data, mode):
        if mode == 'function_name':
            get_function_name = lambda test_example: regex.match('assert [\(]*([^\(]+)\(', test_example).group(1)
            info = [get_function_name(x['test_list'][0]) for x in raw_data]
        elif mode == 'assertion':
            info = [x['test_list'][0] for x in raw_data]
        elif mode == 'assertion-full':
            info = [x['test_list'] for x in raw_data]
        else:
            raise Exception(f'Mode {mode} not supported.')
        nls = [x['text'] for x in raw_data]
        codes = [x['code'] for x in raw_data]
        return list(zip(nls, codes, info))






In [3]:
class Object(object):
  pass



In [4]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

import argparse
import copy
import json

import os
import pickle
import random
import signal
import tensorflow as tf
import time
from glob import glob
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score as m
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
from rouge_metric import PyRouge
import nltk


def codex_with_info(configs, dataset, prefixes, tokenizer, model):

    def codex_greedy(prompt):
        inputs = tokenizer(text=prompt, truncation=True,
                           return_tensors="pt").input_ids

        sample = model.generate(inputs, temperature=0, output_scores=True,
                                return_dict_in_generate=True, max_new_tokens=200, renormalize_logits=True)

        output = tokenizer.decode(sample[0][0])

        
        # return this to get text
        text = output[len(prompt):].split("</code>", 1)[0]
        #print(text)
        return text, None, None
    

    def codex_sample(prompt):
        inputs = tokenizer(text=prompt, truncation=True,
                           return_tensors="pt").input_ids

        sample = model.generate(inputs, temperature=configs.temperature, output_scores=True,
                                return_dict_in_generate=True, do_sample=True, max_new_tokens=200, renormalize_logits=True)

        output = tokenizer.decode(sample[0][0])

        # return this to get text
        text = output[len(prompt):].split("</code>", 1)[0]
        # return this to return token ids
        encoded = tokenizer.encode(text)
        enc_len = len(encoded)

        log_probs = np.zeros(enc_len)
        # return this as final logs
        for i in range(enc_len):
            log_probs[i] = sample[1][i][0][encoded[i]]
        #print("prediction:")
        #print(text)
        #print("ids of predicted")
        #print(encoded)
        #print("log_probs")
        #print(log_probs)
        return text, encoded, log_probs.tolist()


    prompt_prefix = ''.join([configs.prompt_template.format(
        src=x[0], trg=x[1], info=x[2]) for x in prefixes])

    # save folder
    save_dir = f'{configs.output_path}/seed-{configs.seed}/{configs.n_prompts}-shot/{configs.mode}-{configs.temperature}/'
    
    os.system(f'mkdir -p {save_dir}')
    # save configs and prefixes
    if configs.rank == 0:
        with open(f'{save_dir}/prefixes.json', 'w') as fout:
            json.dump(prefixes, fout)
            fout.close()
        with open(f'{save_dir}/configs.pkl', 'wb') as fout:
            pickle.dump(configs, fout)
            fout.close()
    ofname = f'{save_dir}/{configs.split}-{configs.rank}.jsonl'
    # load checkpoint
    if os.path.exists(ofname):
        n_processed_examples = len(open(ofname).readlines())
    else:
        n_processed_examples = 0
    pbar = tqdm(dataset)

    with open(ofname, 'a') as fout:
        for i, (src, trg, info) in enumerate(pbar):
            if i < n_processed_examples:
                continue
            prompt = prompt_prefix + \
                configs.example_template.format(src=src, info=info)
            #print("prompt")
            #print(prompt)
            while True:
                try:
                    trg_prediction, tokens, logprobs = codex_greedy(
                        prompt) if configs.mode == 'greedy' else codex_sample(prompt)
                    time.sleep(2)
                    break
                except Exception as e:
                    
                    print(e, flush=True)
                    time.sleep(3)
            try:

                bleu_score = sentence_bleu([[ch for ch in trg]], [
                                           ch for ch in trg_prediction])
                #print("bleu score:")
                #print(bleu_score)
            except:
                bleu_score = 0
            
            try: 

                rouge_score = 0
                rouge = PyRouge(rouge_n=(1,), rouge_l=True,
                                    rouge_w=True, rouge_w_weight=1.2, rouge_s=True)
                    
                rouge_score = rouge.evaluate_tokenized(
                        [[ch for ch in trg_prediction]], [[ch for ch in trg]])
               

            except:
                print("fail rouge")
                rouge_score = 0

            try: 
                
                
                # should work in colab
                meteor_score = m([ch for ch in trg], [ch for ch in trg_prediction])
                
                

            except: 
               
               
                print("fail meteor")
                meteor_score = 0

            print(
                json.dumps(
                    {
                        'prompt': prompt,
                        'src': src,
                        'trg_prediction': trg_prediction,
                        'reference': trg,
                        'tokens': tokens,
                        'logprobs': logprobs,
                        'bleu': bleu_score,
                        'rouge': rouge_score, 
                        'meteor': meteor_score
                        
                    }
                ),
                file=fout, flush=True
            )
            pbar.set_description(f'Process {configs.rank}')
        fout.close()






  


class CollectorWithInfo(object):
    def __init__(self, dataset, tokenizer, model, rank):
       #change here for arguments that would be command line arguments
      self.configs = Object()
      self.configs.output_path = "/content/drive/MyDrive/data/mbr-2B/mbpp"
      self.configs.split = "test"
      self.configs.seed = [1,2,3,4,5]
      self.configs.temperature = [0.9]
      self.configs.n_prompts = [3]
      self.configs.n_samples = 5 # number of seeds multiplied by number of samples is the total number that we can look at
      self.configs.mode = "sample"
      self.configs.prompt_template = "<info>{info}</info>\n<text>{src}</text>\n<code>{trg}</code>\n"
      self.configs.example_template = "<info>{info}</info>\n<text>{src}</text>\n<code>"
      self.configs.end_template = "</code>"
      self.configs.shuffle_prefix = False
      self.configs.saved_prefixes_path_template = None
      self.configs.rank = rank
      self.dataset = dataset
      self.tokenizer = tokenizer
      self.model = model

    def __call__(self):

        
        configs = copy.deepcopy(self.configs)
        
        for seed in self.configs.seed:
            for n_prompts in self.configs.n_prompts:
                for temperature in self.configs.temperature:
                    configs.n_prompts = n_prompts
                    configs.seed = seed
                    configs.temperature = temperature
                    random.seed(configs.seed)
                    if configs.saved_prefixes_path_template is not None:
                        prefix_pool = list()
                        for path in glob(configs.saved_prefixes_path_template, recursive=True):
                            prefix_pool.extend(json.load(open(path)))
                        prefix_pool = sorted(
                            set([tuple(x) for x in prefix_pool]))
                        prefixes = random.sample(
                            prefix_pool, configs.n_prompts)
                    else:
                        prefixes = random.sample(
                            self.dataset.data['train'], configs.n_prompts)
                    if configs.shuffle_prefix:
                        original_prefixes = copy.deepcopy(prefixes)
                        while original_prefixes == prefixes:
                            random.shuffle(prefixes)
                    codex_with_info(
                        configs, self.dataset.data[configs.split], prefixes, self.tokenizer, self.model)

    

In [ ]:
# only run this code if you are collecting data from codegen
import torch # this line and next switch to GPU
torch.set_default_tensor_type(torch.cuda.FloatTensor)
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
model = AutoModelForCausalLM.from_pretrained(
        "Salesforce/codegen-2B-mono")

In [ ]:
# collect data from codegen
mbpp = MBPPGoogleDataset(mode = 'assertion')
num_rank = [0]
for i in num_rank:
  newcollector = CollectorWithInfo(mbpp, tokenizer, model, i)
  newcollector()

  0%|          | 0/500 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:   0%|          | 1/500 [00:24<3:23:29, 24.47s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:   0%|          | 2/500 [00:45<3:08:24, 22.70s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:   1%|          | 3/500 [01:19<3:49:54, 27.76s/it]The attention mask and the p

tuple index out of range


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  24%|██▍       | 119/500 [45:33<3:02:22, 28.72s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  24%|██▍       | 120/500 [46:00<2:58:25, 28.17s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  24%|██▍       | 121/500 [46:25<2:51:12, 27.10s/it]The attention mask and the pad token id were not set. As a co

tuple index out of range


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  32%|███▏      | 162/500 [1:01:34<2:38:24, 28.12s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  33%|███▎      | 163/500 [1:01:55<2:25:34, 25.92s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  33%|███▎      | 164/500 [1:02:17<2:18:40, 24.76s/it]The attention mask and the pad token id were not set. A

tuple index out of range


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  41%|████      | 206/500 [1:18:16<2:17:16, 28.02s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  41%|████▏     | 207/500 [1:18:38<2:07:01, 26.01s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Process 0:  42%|████▏     | 208/500 [1:19:02<2:04:58, 25.68s/it]The attention mask and the pad token id were not set. A

In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Copyright (c) Meta Platforms, Inc. and affiliates.


import json
import os
import pickle
import regex
import signal
import subprocess
import tempfile
import threading
from datasets import load_metric
from glob import glob
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm





class Command(object):
    def __init__(self, cmd):
        self.cmd = cmd
        self.process = None

    def run(self, timeout):
        def target():
            self.process = subprocess.Popen(self.cmd, shell=True, preexec_fn=os.setsid)
            self.process.communicate()

        thread = threading.Thread(target=target)
        thread.start()

        thread.join(timeout)
        if thread.is_alive():
            os.killpg(self.process.pid, signal.SIGTERM)
            thread.join()
        return self.process.returncode


class PythonFunctionExecutor(object):
    def __init__(self, function_content, function_call, timeout=10):
        self.function_content = function_content
        self.function_call = function_call
        self.timeout = timeout

    def __call__(self):
        tempdir = tempfile.TemporaryDirectory()
        with open(f'{tempdir.name}/code.py', 'w') as fout:
            print(self.function_content, file=fout)
            print(f'result = {self.function_call}', file=fout)
            print(f'import pickle', file=fout)
            print(f'pickle.dump(result, open("{tempdir.name}/execution_result.pkl", "wb"))', file=fout)
        command = Command(f'python {tempdir.name}/code.py >/dev/null 2>&1')
        execution_status = command.run(timeout=self.timeout)
        if execution_status == 0:
            try:
                execution_results = pickle.load(open(f'{tempdir.name}/execution_result.pkl', 'rb'))
            except:
                execution_results = None
        else:
            execution_results = None
        tempdir.cleanup()
        return execution_status, execution_results


def execute_mbpp_google_folder(base_path):
    # single assertion
    
    dataset = MBPPGoogleDataset(mode='assertion')
    
    
    
    for path in glob(f'{base_path}/*jsonl'):  # execute first assertion call
        
        if os.path.exists(path.replace('jsonl', 'exec.pkl')):
            continue
        split = os.path.basename(path).split('-')[0]
        execution_results = list()
        for i, line in enumerate(tqdm(open(path).readlines())):
            assertion = dataset.data[split][i][-1]
            command = regex.match(f'assert (.+)==.+', assertion).group(1)
            item = json.loads(line)
            python_function = item['trg_prediction']
            executor = PythonFunctionExecutor(python_function, command)
            execution_result = executor()
            execution_results.append(execution_result)
        with open(path.replace('jsonl', 'exec.pkl'), 'wb') as fout:
            pickle.dump(execution_results, fout)
    # multiple assertions (cheating)
    dataset = MBPPGoogleDataset(mode='assertion-full')
    for path in glob(f'{base_path}/*jsonl'):  # execute all assertion calls
        if os.path.exists(path.replace('jsonl', 'execfull.pkl')):
            continue
        split = os.path.basename(path).split('-')[0]
        execution_results = list()
        for i, line in enumerate(tqdm(open(path).readlines())):
            execution_result = list()
            item = json.loads(line)
            python_function = item['trg_prediction']
            for assertion in dataset.data[split][i][-1]:
                command = regex.match(f'assert (.+)==.+', assertion).group(1)
                executor = PythonFunctionExecutor(python_function, command)
                execution_result.append(executor())
            execution_results.append(execution_result)
        with open(path.replace('jsonl', 'execfull.pkl'), 'wb') as fout:
            pickle.dump(execution_results, fout)
    # multiple assertions (pass or fail)
    for path in glob(f'{base_path}/*jsonl'):
        if os.path.exists(path.replace('jsonl', 'execfullpass.pkl')):
            continue
        split = os.path.basename(path).split('-')[0]
        execution_results = list()
        for i, line in enumerate(tqdm(open(path).readlines())):
            execution_result = list()
            item = json.loads(line)
            python_function = item['trg_prediction']
            for assertion in dataset.data[split][i][-1]:
                command = regex.match(f'assert (.+==.+)', assertion).group(1)
                executor = PythonFunctionExecutor(python_function, f'({command})')
                execution_result.append(executor())
            execution_results.append(execution_result)
        with open(path.replace('jsonl', 'execfullpass.pkl'), 'wb') as fout:
            pickle.dump(execution_results, fout)





In [7]:
!pip install datasets
!pip install data



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for data: filename=data-0.4-py3-none-any.whl size=7244 sha256=af9bc73875557e3997b4c3fd7ef4db36b50948745297d33bdd14a6a2c538f16f
  Stored in directory: /root/.cache/pip/wheels/d0/e8/fa/e253c256048ea58d99a8abb5e751abb6a838af6f12887b5418
Successfully built data


In [8]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

import os
import tempfile
from datasets import load_metric
from tqdm import tqdm





""" dataset keys: src, trg_prediction, reference """


def evaluate_charbleu(dataset):
    
    bleu = load_metric('bleu')
    predictions = [[ch for ch in item['trg_prediction']] for item in dataset]
    references = [[[ch for ch in item['reference']]] for item in dataset]
    return bleu.compute(predictions=predictions, references=references)


""" dataset keys: src, trg_prediction, reference (only trg_prediction useful) """




def evaluate_google_mbpp(dataset, reference_path, split='test', timeout=10, return_details=False):
    references = MBPPGoogleDataset(reference_path)

    assert len(dataset) == len(references.raw_data[split])
    tempdir = tempfile.TemporaryDirectory()
    passed_information = list()
    pbar = tqdm(references.raw_data[split])
    for i, item in enumerate(pbar):
        if 'execution_result_full_pass' in dataset[i]:
            passed_information.append(
                int(all(x[1] == True for x in dataset[i]['execution_result_full_pass'])))
        else:
            test_cases = item['test_list']
            test_setups = item['test_setup_code']
            code = dataset[i]['trg_prediction']
            # write code to file
            with open(f'{tempdir.name}/code.py', 'w') as fout:
                print(code, file=fout)
                print(test_setups, file=fout)
                for case in test_cases:
                    print(case, file=fout)
                fout.close()
            command = Command(f'python {tempdir.name}/code.py >/dev/null 2>&1')
            execution_result = (command.run(timeout=timeout) == 0)
            passed_information.append(int(execution_result))
        pbar.set_description(f'{sum(passed_information)} out of {i+1} passed.')
    tempdir.cleanup()
    if return_details:
        return passed_information
    else:
        return sum(passed_information) / len(passed_information)


In [10]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# add context-specific linear interpolation 

import collections
import json
import pickle
import numpy as np
import os
import random
from glob import glob
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score as m
from sklearn import preprocessing
from rouge_metric import PyRouge
import string



""" k sets of configs in separate paths, n * k choose 1 selector """


class MultiSampleSelector(object):
    def __init__(self, paths, split='dev', interpolation = False):
        self.paths = list(sorted(glob(paths, recursive=True))) if isinstance(
            paths, str) else list(sorted(paths))
        self.split = split
        self.data = collections.defaultdict(list)
        self.args = collections.defaultdict(list)
        self.hyper_params = np.array([1.0, 100.0, 0.3])  # bleu, mbr-exec, avg_log, meteor, rouge

        for i, path in enumerate(self.paths):
            self.args[i] = pickle.load(
                open(f'{self.paths[0]}/configs.pkl', 'rb'))
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.jsonl'):
                self.data[i, idx].extend(
                    [json.loads(x) for x in open(f'{path}/{split}-{idx}.jsonl')])
                idx += 1
        for path_id, sample_id in self.data:
            for item in self.data[path_id, sample_id]:
                try:
                    avg_logprob, sum_logprob = self.extract_logprob_stats(
                        item, path_id)
                    item['avg_logprob'] = avg_logprob
                    item['sum_logprob'] = sum_logprob

                except:
                    item['avg_logprob'] = item['sum_logprob'] = 0
                
    def extract_logprob_stats(self, item, path_id):
        current_seq = ''
        extracted_position = None
        for i, _ in enumerate(item['tokens']):
            current_seq += item['tokens'][i]
            if current_seq.find(item['trg_prediction']) != -1 and current_seq.find(self.args[path_id].end_template) != -1:
                extracted_position = i + 1
                break
        logprobs = item['logprobs'][:extracted_position] if extracted_position is not None else item['logprobs']
        # handle potential codex bug on positive log probability
        logprobs = list(filter(lambda x: x <= 0 and x != float('-inf'), logprobs)) #keep 0 log probs because they indicate prob = 1 and remove -inf log probs
        return np.mean(logprobs), np.sum(logprobs)

    def select(self, ids=None, key_extractor=lambda x: x['avg_logprob'], return_keys=False, interpolation = False):
        if ids is None:
            ids = self.data.keys()
        ids = list(sorted(ids))
        print(f'Selecting Samples from IDs: {ids}', flush=True)
        n_examples = len(self.data[ids[0]])
        selected_examples = list()
        sample_keys = collections.defaultdict(list)
        for i in range(n_examples):
            max_key = None
            selected_item = None
            for idx in ids:
                item = self.data[idx][i]
                key = key_extractor(item)
                sample_keys[idx].append(key)
                if max_key is None or key > max_key:
                    max_key = key
                    selected_item = item
            assert selected_item is not None
            selected_examples.append(selected_item)
        if return_keys:
            return selected_examples, sample_keys
        else:
            return selected_examples


class ExecutionBasedMultiSampleSelector(MultiSampleSelector):
    def __init__(self, paths, split='dev', execution_type=None, interpolation = False):
        super().__init__(paths, split=split)
        self.execution_type = execution_type
        for i, path in enumerate(self.paths):
            if execution_type == 'mbpp':
                execute_mbpp_google_folder(path)
            else:
                raise Exception(
                    f'Execution type {execution_type} not supported.')
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.exec.pkl'):
                for j, execution_result in enumerate(pickle.load(open(f'{path}/{split}-{idx}.exec.pkl', 'rb'))):
                    self.data[i, idx][j]['execution_result'] = execution_result
                idx += 1
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.execfull.pkl'):
                for j, execution_result in enumerate(pickle.load(open(f'{path}/{split}-{idx}.execfull.pkl', 'rb'))):
                    self.data[i, idx][j]['execution_result_full'] = execution_result
                idx += 1
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.execfullpass.pkl'):
                for j, execution_result in enumerate(pickle.load(open(f'{path}/{split}-{idx}.execfullpass.pkl', 'rb'))):
                    self.data[i, idx][j]['execution_result_full_pass'] = execution_result
                idx += 1


class IntraMultiSampleSelector(MultiSampleSelector):
    def __init__(self, paths, split='dev', interpolation = False):
        super().__init__(paths, split=split)
        # to normalize log_avg,  just take e^x, to normalize mbr, we just take mbr/argument
        # already try to have it normalized
        # current and max = np array of normalized bleu, mbr, log_avg()

    def greater(self, current, max):  # potentially make contextual

        # diff is array of differences ranging from [-1, 1]

        # multiply by weights in linear interpolation then sum
        mult = np.dot(current - max, self.hyper_params)

        if mult > 0:
            return True
        else:
            return False

    def select(
        self,
        ids=None,
        key_extractor=None,
        second_key_extractor=None,
        return_keys=False, interpolation=False
    ):
        if ids is None:
            ids = self.data.keys()
        elif isinstance(ids, int):
            ids = [(i, j) for i in set(x[0] for x in self.data.keys())
                   for j in range(ids)]
        ids = list(sorted(ids))
        ids_length = len(ids)
        id_set = set(ids)
        sample_keys = collections.defaultdict(list)
        print(f'Selecting Samples from IDs: {ids}')
        n_examples = len(self.data[ids[0]])

        selected_examples = list()
        # 500 examples, where if we have 20 in the id set, we look at each id at self.data[id][length = 500]
        for i in range(n_examples):
            
            max_key = None
            selected_item = None
            for idx in id_set:
                item = self.data[idx][i]

                first_keys = list()
                for grndtruth_idx in ids:
                    # i can be up to 500 and grndtruth_idx is the hyperparameter we choose
                    grndtruth_item = self.data[grndtruth_idx][i]

                    key = key_extractor(item, grndtruth_item)
                    first_keys.append(key)
                # first key is the number that match with it in the sample (up to 20 etc. ) and second value is the log prob
                first_key = sum(first_keys)
                second_key = second_key_extractor(
                    item) if second_key_extractor is not None else 0
                current_key = (first_key, second_key)
                item['mbr_key'] = current_key

                # create interpolation value which are the values normalized from 0 to 1
                item['interpolation'] = np.array(
                    [item['bleu'], first_key/ids_length, np.exp(item['avg_logprob'])])
                if interpolation:

                    current_key = (item['interpolation'],)
                    # now need to write the case where interpolation = true
                    sample_keys[idx].append(current_key)
                    if max_key is None or self.greater(current_key[0], max_key[0]):
                        max_key = current_key
                        selected_item = item
                else:
                    sample_keys[idx].append(current_key)
                    # should try ranking system but must ensure that rank 1 is actually given as n = size
                    if max_key is None or current_key > max_key:
                        max_key = current_key
                        selected_item = item
            assert selected_item is not None
            selected_examples.append(selected_item)

        if return_keys:
            return selected_examples, sample_keys
        else:
            return selected_examples


class ExecutionBasedIntraMultiSampleSelector(IntraMultiSampleSelector):
    def __init__(self, paths, split='dev', execution_type=None, interpolation = False):
        super().__init__(paths, split=split)
        self.execution_type = execution_type
        for i, path in enumerate(self.paths):
            if execution_type == 'mbpp':
                execute_mbpp_google_folder(path)
            else:
                raise Exception(
                    f'Execution type {execution_type} not supported.')
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.exec.pkl'):
                for j, execution_result in enumerate(pickle.load(open(f'{path}/{split}-{idx}.exec.pkl', 'rb'))):
                    self.data[i, idx][j]['execution_result'] = execution_result
                idx += 1
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.execfull.pkl'):
                for j, execution_result in enumerate(pickle.load(open(f'{path}/{split}-{idx}.execfull.pkl', 'rb'))):
                    self.data[i, idx][j]['execution_result_full'] = execution_result
                idx += 1
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.exec.codexcases.pkl'):
                for j, execution_result in enumerate(pickle.load(open(f'{path}/{split}-{idx}.exec.codexcases.pkl', 'rb'))):
                    self.data[i, idx][j]['execution_result_codexexec'] = execution_result
                idx += 1
            idx = 0
            while os.path.exists(f'{path}/{split}-{idx}.execfullpass.pkl'):
                for j, execution_result in enumerate(pickle.load(open(f'{path}/{split}-{idx}.execfullpass.pkl', 'rb'))):
                    self.data[i, idx][j]['execution_result_full_pass'] = execution_result
                idx += 1


"""equivalence checking functions"""
# base equavalence checking function


def single_exec_result_matching(exec_x, exec_y, good_execution_result):
    try:
        if exec_x[0] == good_execution_result and exec_y[0] == good_execution_result and exec_x[1] == exec_y[1]:
            return 1
        else:
            return 0
    except:
        return 0


# first assertion call matching
def execution_selection_function(x, y, good_execution_result=0):
    exec_x, exec_y = x['execution_result'], y['execution_result']
    return single_exec_result_matching(exec_x, exec_y, good_execution_result)


# just executability checking
def executability_selection_function(x, good_execution_result=0):
    exec_res = x['execution_result']
    return exec_res[0] == good_execution_result


def bleu_selection_function(x, y):
    return sentence_bleu([[ch for ch in x['trg_prediction']]], [ch for ch in y['trg_prediction']])


def token_bleu_selection_function(x, y):
    return sentence_bleu([x['trg_prediction'].split()], y['trg_prediction'].split())

def token_meteor_selection_function(x,y): 
    return m(x['trg_prediction'].split(), y['trg_prediction'].split()) 

def token_rouge_selection_function(x,y): 
    rouge = PyRouge(rouge_n = 1, rouge_l = True)

    return rouge.evaluate_tokenized([x['trg_prediction'].split()], [y['trg_prediction'].split()])['rouge-1']['r']

def meteor_selection_function(x,y): 
    return m([ch for ch in x['trg_prediction']], [ch for ch in y['trg_prediction']]) 
def rouge_selection_function(x,y): 
    rouge = PyRouge(rouge_n = 1, rouge_l = True)

    return rouge.evaluate_tokenized(
        [[ch for ch in x['trg_prediction']]], [[ch for ch in y['trg_prediction']]])['rouge-1']['r']

def bash_execution_tokenbleu_selection_function(x, y):
    if not x['executable'] or not y['executable']:
        return 0
    x = x['trg_prediction_splitted']
    y = y['trg_prediction_splitted']
    return sentence_bleu([x], y)
# balls
def contextual_log_probs(x):
    count = 0
    total = 0 # penalize longer outputs since it likely drags on (every 8 characters)
    sum = 0
    for i in (x['logprobs']):
      if x['tokens'][count] != '</' and x['tokens'][count] != 'code' and x['tokens'][count] != '>':
        if x['tokens'][count].isalpha(): #only look at log probs if alpha (only letters)
          sum+= (1 * i)
        else:
          if x['tokens'][count] in string.punctuation: #punctuation
            sum += (0 * i)
          else: 
            sum += (10 * i) #numbers
      else: 
        total -= 1
      count += 1
      total += 1
      if count % 10 == 0 and count > 1: # rewards shorter code
        total += 4
        
         
     

        

       
    
    return sum/total


"""
    select and evaluate a group in batch
    required keys:
        data_split: 'train', 'dev' or 'test'
        temperature: 0.1 .. 1.0
        criterion: 'mbr_exec' ... see full options in the function
        data_path: root data path for the task
        n_samples: number of candidates
        rand_seed: random seed for one experiment
"""


def select_mbpp(args, return_selected=False, return_selector=False):
    data_split, temperature, criterion, data_path, n_samples, rand_seed = args
    mbpp_good_execution_result = 0
    data_path = f'{data_path}/seed-*/**/*-{temperature}/'
    secondary_key_function = None
    interpolation = False
    if criterion == 'mbr_exec':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')

        def sample_selection_function(x, y): return execution_selection_function(
            x, y, mbpp_good_execution_result)

        def secondary_key_function(x): return x['sum_logprob']
    
    elif criterion == 'interpolation_mbr_exec':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')

        def sample_selection_function(x, y): return execution_selection_function(
            x, y, mbpp_good_execution_result)  # this can change because this is just mbr based on execution result -> add new ones that use different functions
        interpolation = True

    elif criterion == 'logprob':
        selector = ExecutionBasedMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(x): return x['sum_logprob']
    elif criterion == 'avg_logprob':
        selector = ExecutionBasedMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(x): return x['avg_logprob']
    elif criterion == 'mbr_bleu':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(
            x, y): return bleu_selection_function(x, y)
    elif criterion == 'context':
        selector = ExecutionBasedMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(x): return contextual_log_probs(x)

    elif criterion == 'mbr_meteor': 
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(
            x, y): return meteor_selection_function(x, y)
    elif criterion == 'mbr_tokenmeteor':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(
            x, y): return token_meteor_selection_function(x, y)
    elif criterion == 'mbr_rouge':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(
            x, y): return rouge_selection_function(x, y) 
    elif criterion == 'mbr_tokenrouge': 
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(
            x, y): return token_rouge_selection_function(x, y)
    elif criterion == 'mbr_tokenbleu':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(
            x, y): return token_bleu_selection_function(x, y)
    elif criterion == 'executability-logprob':
        selector = ExecutionBasedMultiSampleSelector(
            data_path, data_split, 'mbpp')

        def sample_selection_function(x): return (executability_selection_function(
            x, mbpp_good_execution_result), x['sum_logprob'])
    elif criterion == 'executability-avglogprob':
        selector = ExecutionBasedMultiSampleSelector(
            data_path, data_split, 'mbpp')

        def sample_selection_function(x): return (executability_selection_function(
            x, mbpp_good_execution_result), x['avg_logprob'])
    elif criterion == 'executability-mbr_bleu':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation

        def sample_selection_function(x, y): return bleu_selection_function(x, y) * (1 - x['execution_result'][0]) * \
            (1 - y['execution_result'][0])
    elif criterion == 'executability-mbr_tokenbleu':
        selector = ExecutionBasedIntraMultiSampleSelector(
            data_path, data_split, 'mbpp')  # pre-execution for faster evaluation
        def sample_selection_function(x, y): return token_bleu_selection_function(x, y) * (1 - x['execution_result'][0]) * \
            (1 - y['execution_result'][0])
    else:
        raise ValueError(f'Unknown criterion: {criterion}')
    id_keys = list(selector.data.keys())
    random.seed(rand_seed)
    ids = random.sample(id_keys, n_samples)

    if secondary_key_function is not None:
        selected = selector.select(
            ids, sample_selection_function, secondary_key_function, interpolation=interpolation)
    else:
        selected = selector.select(
            ids, sample_selection_function, interpolation=interpolation)
    if return_selector:
        return selector
    elif return_selected:
        return selected
    else:

        result = evaluate_google_mbpp(selected, '/content/drive/MyDrive/data/mbpp/mbpp.jsonl', 'test')
        return result






In [14]:
#wherever you have the codex or codegen data saved (you can use our shared drive)
codegen_location = '/content/drive/Shareddrives/COS484FinalProject/data/mbr-2B/mbpp'
codex_location = '/content/drive/Shareddrives/COS484FinalProject/codexData/mbpp/mbpp'
select_mbpp(('test', 0.3, 'logprob',
            codegen_location, 10 , 0)) # (test/train/validation, temperature, decode scheme, location of generated code, sample size, seed )












Selecting Samples from IDs: [(0, 2), (3, 1), (3, 4), (4, 2), (4, 4), (5, 0), (5, 1), (6, 1), (6, 2), (6, 3)]


135 out of 500 passed.: 100%|██████████| 500/500 [00:00<00:00, 576.14it/s]


0.27